In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split

In [ ]:
# Nombre del archivo CSV
path_csv = '../data/csv/history.csv'

# Listas para almacenar los valores de las columnas 's' y 'reward'
sy_values = []
sx_values = []
a_values = []
sy1_values = []
sx1_values = []
reward_values = []

In [ ]:
# Leer el archivo CSV
with open(path_csv, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Quitar la primera fila donde están los nombres de las columnas

    for row in csv_reader:
        step, y, x, action, next_y, next_x, reward, done =  row
        sy_values.append(float(y))
        sx_values.append(float(x))
        a_values.append(float(action))
        sy1_values.append(float(next_y))
        sx1_values.append(float(next_x))
        reward_values.append(float(reward))

In [ ]:
# Convertir las listas en NumPy arrays
sy_array = np.array(sy_values)
sx_array = np.array(sx_values)
a_array = np.array(a_values)
sy1_array = np.array(sy1_values)
sx1_array = np.array(sx1_values)
reward_array = np.array(reward_values)

In [ ]:
input_data1 = np.column_stack((sy_array, sx_array, a_array))
target_data1 = np.column_stack((sy1_array, sx1_array))
input_data2 = input_data1
target_data2 = reward_array

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(input_data1, target_data1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(input_data2, target_data2, test_size=0.2, random_state=42)

In [ ]:
oculta1 = keras.layers.Dense(units=48, input_shape=(3,), activation='relu')
oculta2 = keras.layers.Dense(units=100, activation='relu')
oculta3 = keras.layers.Dense(units=50, activation='relu')
salida1 = keras.layers.Dense(units=2)
salida2 = keras.layers.Dense(units=1)

modelo1 = keras.Sequential([oculta1, salida1])
modelo2 = keras.Sequential([oculta1, salida2])

modelo1.compile(optimizer='adam', loss='mean_squared_error')
modelo2.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
historial1 = modelo1.fit(X_train1, y_train1, epochs=50, batch_size=32, verbose=False, validation_data=(X_test1, y_test1))
print("Modelo entrenado!")

In [ ]:
test_loss1 = modelo1.evaluate(X_test1, y_test1)
print(f'Test loss: {test_loss1}')

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de pérdida")
plt.plot(historial1.history["loss"])

In [ ]:
modelo1.save('../data/models/_modelo_entorno.h5')
#El nombre empieza con _ para no sobreescribir los modelos ya creados

In [ ]:
for k in range(2):
    for i in range(4):
        #print("Hagamos una predicción!")
        test_input = np.array([[4,k,i]])
        resultado = modelo1.predict(test_input)
        print("El vector de entrada [y,x,a] es ",[4,k,i])
        print("El resultado es " + str(resultado) + "-> Redondeo:" + str(round(resultado[0][0]))+ ',' + str(round(resultado[0][1])))


Red para modelar la función de recompensa:

In [ ]:
historial2 = modelo2.fit(X_train2, y_train2, epochs=20, batch_size=32, verbose=False, validation_data=(X_test2, y_test2))
print("Modelo entrenado!")

In [ ]:
test_loss2 = modelo2.evaluate(X_test2, y_test2)
print(f'Test loss: {test_loss2}')

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de pérdida")
plt.plot(historial2.history["loss"])

In [ ]:
for k in range(3,5):
    for i in range(4):
        #print("Hagamos una predicción!")
        test_input = np.array([[1,k,i]])
        resultado = modelo2.predict(test_input)
        print("El vector de entrada [y,x,a] es ",[1,k,i])
        print("El resultado es " + str(resultado[0]) + "-> Redondeo:" + str(round(resultado[0][0])))

In [ ]:
modelo2.save('../data/models/_modelo_reward.h5')
#El nombre empieza con _ para no sobreescribir los modelos ya creados